<center><h1>Code Python</h1></center>

## Importation des librairies

In [1]:
# Pandas
import pandas as pd
# Contenant l'algorithme du Truth Finder
import TruthFinder as tf 
# Contenant le partitionnement par force bruite appliqué avec le Truth Finder(Pour le moment)
import AccuGenPartition as agp 
import DepenAccuAccuSim as daas 

In [2]:
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

In [3]:
import TDAC as aac


In [4]:
#import AccuPartitionClustering as apc

## Importation des données

In [5]:
dataframe = pd.read_csv("data_semi/62_attributs/domaine25/data.csv")
dataframe = pd.read_csv("stock/stock_clean_2011-07-01.csv")

dataframe = pd.read_csv("examens/62_attributs/data.csv")

dataframe = pd.read_csv("flight/flight_clean_2011-12-01.csv")

#dataframe = pd.read_csv("data_conf22/data.csv")
#dataframe['ID'] = [i for i in range(dataframe.shape[0])]
#dataframe = dataframe[['ID','Object','Property','Value','Source']]
dataframe.head(1)

,Unnamed: 0,Object,Property,Value,Source
0,0,UA-2708-EWR-CLT,Scheduled departure,2011-12-01 2:55 PM,ua


In [6]:
#dataframe.to_csv('dataconf1/data.csv',index=False)
#data_truth.to_csv('dataconf1/data_truth.csv',index=False)

In [7]:
#dataframe[['Object','Property','Value']].drop_duplicates().groupby(by=['Object','Property'])
#for key, value_of_di in dataframe[['Object','Property','Value']].drop_duplicates().groupby(by=['Object','Property']):
    #print(key[0]+key[1])


## Importation des données vrai pour le calcule de précision avec la fonction __oracle__

In [8]:
data_truth = pd.read_csv("data_semi/124_attributs/domaine25/data_truth.csv")


data_truth = pd.read_csv("examens/124_attributs/data_truth.csv")

#data_truth = pd.read_csv("stock/stock_truth_2011-07-01.csv")
#data_truth = pd.read_csv("data_conf22/data_truth.csv")
data_truth = pd.read_csv("flight/flight_truth_2011-12-01.csv")

#data_truth = data_truth[['Object','Property','Value']]
data_truth.head(1)

,Object,Property,Value
0,AA-1221-MCO-ORD,Scheduled departure,2011-12-01 8:00 PM


In [9]:
dataframe = dataframe.drop_duplicates(subset=['Object', 'Property','Value','Source'], keep="first")

## Execution de MajorityVote sur les données

In [17]:
def majorityVote(dataframe,data_truth):
    dataframe["Object"]=[str(x) for x in dataframe["Object"]]
    dataframe["Property"]=[str(x) for x in dataframe["Property"]]
    dataframe["ObjectProperty"] = dataframe["Object"]+dataframe["Property"]
    resultat = pd.DataFrame(columns=['Object','Property','Value'])
    u=0
    for o_p in dataframe["ObjectProperty"].unique():
        indices = dataframe["ObjectProperty"] == o_p
        df = dataframe.loc[indices]
        dic = dict(df.Value.value_counts())
        max_ = 0
        key = 0
        #print(dic)
        for i in range(len(dic)):
            if list(dic.values())[i] > max_:
                max_ = list(dic.values())[i]
                key = list(dic.keys())[i]
        resultat.loc[u] = [list(df.Object)[0],list(df.Property)[0],key]
        u+=1
        
    data_truth1 = tf.get_truth_to_dict(data_truth)
    resultat = tf.get_truth_to_dict(resultat)
    evaluation_r = tf.evaluation(data_truth1,resultat,dataframe)
    
    return evaluation_r

In [18]:
%%time
majorityVote(dataframe,data_truth)

CPU times: user 1min 1s, sys: 879 ms, total: 1min 1s
Wall time: 1min 15s


{'precision': 0.667442324313543,
 'recall': 0.6022918166546676,
 'accuracy': 0.80615,
 'f1_score': 0.6331956226938724}

## Execution de Truth Finder sur les données

L'algorthme du Truth Finder que nous avons implémenté tient compte des données dont les valeurs sont continue d'une part et catégoriel d'autre. 

Il retourne une liste contenant respectivement : __les données de départ ajouter de deux nouvelles colonne (fiabilité des source et confidence des valeurs)__ , __Nombre itération__ , __Max accuracy__ , __Avg accuracy__ et __Oracle accuracy__ .

In [50]:
%%time
out_tf = tf.train(dataframe,
                  initial_trustworthiness=0.8,
                  max_iterations=20,
                  threshold=1e-6,
                  data_truth=data_truth,
                  rho=0.5,
                  afficher=True)

---- Info sur data ----- 
Nombre object :  100
Nombre attribue :  6
Nombre source :  37
Nombre observation :  8644
---- ------------ ----- 

End sim
---- Itération :  1 ----- 
---- Itération :  2 ----- 
---- Itération :  3 ----- 
---- Resultat----- 
Nombre itération :  3
{'precision': 0.9003816027314722, 'recall': 0.8594708588957055, 'accuracy': 0.8578204534937529, 'f1_score': 0.8794507111329083}
CPU times: user 20.5 s, sys: 209 ms, total: 20.7 s
Wall time: 22.3 s


## Execution de DEPEN sur les données

In [20]:
%%time
out_depen = daas.train(dataframe, max_iterations=20,
          threshold=1e-6,initial_trustworthiness=0.8,c=0.2,n=50,alpha=0.2,rho=0.5,afficher=True,algo="DEPEN",data_truth=data_truth)

---- Info sur data ----- 
Nombre object :  1000
Nombre attribue :  6
Nombre source :  10
Nombre observation :  60000
---- ------------ ----- 

---- Resultat----- 
Nombre itération :  3
{'precision': 0.6115284887972874, 'recall': 0.5518358531317494, 'accuracy': 0.7781166666666667, 'f1_score': 0.5801507458450283}
CPU times: user 19min 43s, sys: 27.3 s, total: 20min 11s
Wall time: 24min 52s


## Execution de ACCU sur les données

In [52]:
%%time
out_depen = daas.train(dataframe, max_iterations=20,
          threshold=1e-6, initial_trustworthiness=0.8,c=0.2,n=50,alpha=0.2,rho=0.5,afficher=True,algo="ACCU",data_truth=data_truth)

---- Info sur data ----- 
Nombre object :  100
Nombre attribue :  6
Nombre source :  37
Nombre observation :  8644
---- ------------ ----- 

---- Resultat----- 
Nombre itération :  7
{'precision': 0.9680658766820647, 'recall': 0.9582504970178927, 'accuracy': 0.9573114298935678, 'f1_score': 0.963133180137876}
CPU times: user 6min 4s, sys: 2.63 s, total: 6min 7s
Wall time: 6min 25s


## Execution de ACCUSIM sur les données

In [22]:
%%time
out_depen = daas.train(dataframe, max_iterations=20, 
          threshold=1e-6, initial_trustworthiness=0.8,c=0.2,n=50,alpha=0.2,rho=0.5,afficher=True,algo="ACCUSIM",data_truth=data_truth)

---- Info sur data ----- 
Nombre object :  1000
Nombre attribue :  6
Nombre source :  10
Nombre observation :  60000
---- ------------ ----- 

---- Resultat----- 
Nombre itération :  11
{'precision': 0.7053387407752144, 'recall': 0.662813944770711, 'accuracy': 0.8361833333333333, 'f1_score': 0.6834154668728057}
CPU times: user 1h 30min 16s, sys: 1min 16s, total: 1h 31min 33s
Wall time: 1h 33min


## Execution de l'algorithme de partitionnement d'attribut par force bruite avec Truth Finder / DEPEN / ACCU ou ACCUSIM sur les données
Attention cette exécution prend énormement de temps lorsque le nombre d'attribut est grande 

In [11]:
%%time
# algo= TF ou DEPEN ou ACCU ou ACCUSIM
out_agp = agp.train(dataframe,
                  initial_trustworthiness=0.8,
                  max_iterations=10,
                  threshold=1e-06,algo="ACCU",
                  data_truth=data_truth,
                  afficher=False)

1 - 2 - 3 - 4 - 5 - 6 - 7 - 8 - 9 - 10 - 11 - 12 - 13 - 14 - 15 - 16 - 17 - 18 - 19 - 20 - 21 - 22 - 23 - 24 - 25 - 26 - 27 - 28 - 29 - 30 - 31 - 32 - CPU times: user 7d 9h 36min 35s, sys: 2h 56min 53s, total: 7d 12h 33min 28s
Wall time: 9d 23h 21min 37s


In [13]:
out_agp1 = out_agp 

In [18]:
out_agp = pd.read_csv('AccuGenPartion_Start_Dimanche_13_09_DS2.txt',sep=';',header=None)

out_agp.columns = ['Partition','AccuracyMax','AccuracyAvg','AccuracyOracle','precision','recall','accuracy','f1_score']

In [30]:
out_agp[out_agp['AccuracyMax'] == max(out_agp['AccuracyMax'])].values[0]

array(["[['Property2'], ['Property5', 'Property1', 'Property4', 'Property3', 'Property6']]",
       0.3109654394945225, 0.2738864737107778, 0.8105930524369304,
       0.8105930524369304, 0.7388634721887792, 0.8799, 0.7730679599420545],
      dtype=object)

In [ ]:
out_agp[ (out_agp['AccuracyMax'] == max(out_agp['AccuracyMax'])) & (out_agp['AccuracyAvg'] == max(out_agp['AccuracyAvg']))]

In [24]:
out_agp[out_agp['AccuracyAvg'] == max(out_agp['AccuracyAvg'])].values

array([["[['Property1', 'Property4', 'Property3', 'Property6'], ['Property2'], ['Property5']]",
        0.3109654394945225, 0.2847969118095947, 0.904239862633734,
        0.904239862633734, 0.8674057649667406, 0.94095,
        0.8854399068774856]], dtype=object)

In [32]:
out_agp[out_agp['AccuracyOracle'] == max(out_agp['AccuracyOracle'])].values

array([["[['Property1'], ['Property2', 'Property5'], ['Property3', 'Property6'], ['Property4']]",
        0.3106912848033265, 0.2517853368228506, 0.9929996037511556,
        0.9929996037511556, 0.9855794441531199, 0.9945666666666668,
        0.9892756102375156],
       ["[['Property1'], ['Property2', 'Property5'], ['Property3'], ['Property4'], ['Property6']]",
        0.3106912848033265, 0.2430156039733014, 0.9929996037511556,
        0.9929996037511556, 0.9855794441531199, 0.9945666666666668,
        0.9892756102375156]], dtype=object)

## Execution de l'algorithme de partitionnement d'attribut par clustering (K-mean) avec Truth Finder / DEPEN / ACCU ou ACCUSIM sur les données
Notre proposition

In [11]:
%%time
aac.train(dataframe,initial_trustworthiness=0.8,
                  max_iterations=10,max_iter_part_check=1,
                  threshold=1e-06,algo="TF",
                  data_truth=data_truth,rho=0.5,
                  afficher=False)

End sim
---- Itération :  1 ----- 
---- Itération :  2 ----- 
---- Itération :  3 ----- 
---- Itération :  4 ----- 
---- Itération :  5 ----- 
---- Itération :  6 ----- 
End sim
---- Itération :  1 ----- 
---- Itération :  2 ----- 
---- Itération :  3 ----- 
---- Itération :  4 ----- 
End sim
---- Itération :  1 ----- 
---- Itération :  2 ----- 
---- Itération :  3 ----- 
---- Itération :  4 ----- 
---- Itération :  5 ----- 
---- Itération :  6 ----- 
---- Itération :  7 ----- 
---- Itération :  8 ----- 
Iter :  1 :  [['Actual arrival', 'Actual departure'], ['Arrival gate', 'Departure gate'], ['Scheduled arrival', 'Scheduled departure']]  |  0.8853183370154649  | Time :  24.790601015090942
CPU times: user 25.4 s, sys: 215 ms, total: 25.6 s
Wall time: 25.3 s


{'precision': 0.8853183370154649,
 'recall': 0.8483448806774442,
 'accuracy': 0.8427811198519204,
 'f1_score': 0.8664373464373465}

In [16]:
out_aac[out_aac['AccuracyMax'] == max(out_aac['AccuracyMax'])].values

array([[list([['Property1'], ['Property2'], ['Property3'], ['Property4'], ['Property5'], ['Property6']]),
        0.33391868091568677, 0.2819415119457284, 0.9316666666666666,
        0.9316666666666666, 0.9316666666666666, 0.9781624500665779,
        0.9316666666666665]], dtype=object)

In [17]:
out_aac[out_aac['AccuracyAvg'] == max(out_aac['AccuracyAvg'])].values

array([[list([['Property1', 'Property2', 'Property3', 'Property6'], ['Property4', 'Property5']]),
        0.3319981292288145, 0.29930966837358974, 0.825, 0.825, 0.825,
        0.9440745672436751, 0.825]], dtype=object)

In [18]:
out_aac[out_aac['AccuracyOracle'] == max(out_aac['AccuracyOracle'])].values

array([[list([['Property1', 'Property6'], ['Property2', 'Property3'], ['Property4', 'Property5']]),
        0.3319981292288145, 0.28158956601693436, 0.9433333333333334,
        0.9433333333333334, 0.9433333333333334, 0.9818908122503329,
        0.9433333333333332],
       [list([['Property1', 'Property6'], ['Property2'], ['Property3'], ['Property4', 'Property5']]),
        0.3319981292288145, 0.2735441246566806, 0.9433333333333334,
        0.9433333333333334, 0.9433333333333334, 0.9818908122503329,
        0.9433333333333332]], dtype=object)